In [1]:
# Ricordati di eseguire questa cella con Control+Invio
import sys;
sys.path.append('../../'); 
import jupman;

# Information retrieval - soluzioni

## [Scarica zip esercizi](../../_static/information-retrieval-exercises.zip)

[Naviga file online](https://github.com/DavidLeoni/softpython/tree/master/exercises/information-retrieval)

<div class="alert alert-warning">

**ATTENZIONE**: questo tutorial è pensato per dare un idea rapida dei problemi principali dell'information retrieval ma è ancora incompleto e spesso presenta i concetti in modo eccessivamente semplificato. 

Se l'argomento t'appassiona non mancano buoni libri sul tema!

</div>


L' _information retrieval_ è l'attività che svolgono i sistemi informatici quando devono recuperare _rapidamente_ delle informazioni _rilevanti_ . Perchè questi due aggettivi? 


Al giorno d'oggi molte aziende hanno a che fare con una mole di dati rilevante - i famosi _big data_. Sia i giganti dell'_information_ _technology (_IT_) come Google e Amazon che aziende di dimensioni più modeste per varie ragioni devono fornire ai clienti dei sistemi di ricerca efficienti. Nel caso di Google i clienti vogliono trovare tutte le pagine nel web che contengono un certo testo. Per Amazon, il cliente desidera conoscere tutti i prodotti in una certa categoria, o prodotti _simili_ ad uno già cercato. 



### Che fare


- scompatta lo zip in una cartella, dovresti ottenere qualcosa del genere: 

```

-jupman.py
-altri file ..
-exercises
     |- information-retrieval
         |- information-retrieval-exercise.ipynb     
         |- information-retrieval-solution.ipynb
         |- altri file ..
```

<div class="alert alert-warning">

**ATTENZIONE**: Per essere visualizzato correttamente, il file del notebook DEVE essere nella cartella szippata.
</div>

- apri il Jupyter Notebook da quella cartella. Due cose dovrebbero aprirsi, prima una console e poi un browser. Il browser dovrebbe mostrare una lista di file: naviga la lista e apri il notebook `exercises/information-retrieval/intro-exercise.ipynb`
- Prosegui leggendo il file degli esercizi, ogni tanto al suo interno troverai delle scritte **DA FARE**, che ti chiederanno di scrivere dei comandi Python nelle celle successive. 

<div class="alert alert-warning">

**ATTENZIONE**: Ricordati di eseguire sempre la prima cella dentro il notebook. Contiene delle istruzioni come `import jupman` che dicono a Python quali moduli servono e dove trovarli. Per eseguirla, vedi le seguenti scorciatoie
</div>



Scorciatoie da tastiera:

* Per eseguire il codice Python dentro una cella di Jupyter, premi `Control+Invio`
* Per eseguire il codice Python dentro una cella di Jupyter E selezionare la cella seguente, premi `Shift+Invio`
* Per eseguire il codice Python dentro una cella di Jupyter E creare una nuova cella subito dopo, premi `Alt+Invio`
* Se per caso il Notebook sembra inchiodato, prova a selezionare `Kernel -> Restart`



## Costruiamo il nostro motore di ricerca

Consideriamo in particolare il problema che risolve Google. Supponiamo di avere tante pagine web (tutto il web !): per ogni pagina abbiamo l'indirizzo della pagina (come [http://softpython.readthedocs.org](http://softpython.readthedocs.org) ) e il testo che contiene in formato HTML, per esempio:

```html
<h2>Presentazione</h2>
<p>Il corso SoftPython fornisce un’introduzione al processamento dati usando Python, un linguaggio di programmazione popolare sia nell’industria che nell’ambito della ricerca. Fra le varie cose tratteremo Matplotlib.</p>
```
Se hai guardato il tutorial sull'[estrazione dati](https://softpython.readthedocs.io/it/latest/exercises/extraction/extraction-solution.html) dovresti già avere un'idea di cosa è l'HTML. Se non l'hai fatto niente paura: puoi tranquillamente ignorare le strane sequenze racchiuse tra minore e maggiore come `<h2>`, tieni solo presente che si chiamano _tag HTML_. Parlando in genere, le useremo come esempio del fatto che quando si cerca dentro del testo si possono trovare cose strane o indesiderate, e un sistema di ricerca fatto bene deve essere pronto alle sorprese.

L'utente vuole trovare tutte le pagine contenti un certo testo, per esempio sul sito di SoftPython potrebbe voler cercare dove ci sono spiegazioni su `grafici con matplotlib`

* INPUT: stringa di ricerca (per es: `grafici con matplotlib`
* OUTPUT: tutte le pagine che contengono la stringa (in questo caso `grafici con matplotlib`


Se hai seguito qualcuno dei tutorial sul nostro sito, forse ti potrebbe venire in mente già una o due soluzioni a questo problema.

**DOMANDA**: Prima di proseguire, pensa a 

- come rappresenteresti i dati delle pagine in Python?
- che funzione Python definiresti per fare la ricerca? 
    - quali sarebbero i parametri della funzione? 
    - cosa varia ad ogni chiamata e cosa invece è più 'statico' ?

Non ti preoccupare delle _performance_ - quando si progetta un _algoritmo_ all'inizio conviene capire bene cosa si vuole. Solo dopo aver realizzato un prototipo e nel caso sia troppo lento, ci si comincia a preoccupare delle performance !

### Una prima soluzione

Vediamone qua una possibile soluzione al problema.




#### Rappresentare una pagina

Cominciamo a rappresentare le pagine web. Per farlo, possiamo usare unsemplice dizionario con le chiavi `url` (l'indirizzo web) e `contenuto` per il testo:
    
```python
{
    'url': 'http://softpython.readthedocs.org',
    'contenuto' : '<h2>Presentazione</h2><p>Il corso SoftPython fornisce un’introduzione al processamento dati usando Python, un linguaggio di programmazione popolare sia nell’industria che nell’ambito della ricerca. Fra le varie cose tratteremo Matplotlib.</p>'
}

```




#### Un database per le pagine

Il web ha tante pagine, quindi vorremo mettere la collezione di pagine in contenitore. Ai fini dell'esempio proviamo ad usare una lista. La popoleremo con due pagine, quella [principale](https://softpython.readthedocs.io) e la pagina chiamata [Visualizzazione](https://softpython.readthedocs.io/it/latest/exercises/visualization/visualization-solution.html). Infine assegnamo la lista alla variabile `pagine` (ricordiamo che il testo che mettiamo è semplificato rispetto a quello effettivo che trovate sul sito)


In [2]:
pagine = [
            {
                'url': 'http://softpython.readthedocs.org',
                'contenuto' : '<h2>Presentazione</h2><p>Il corso SoftPython fornisce un’introduzione al processamento dati usando Python, un linguaggio di programmazione popolare sia nell’industria che nell’ambito della ricerca. Fra le varie cose tratteremo Matplotlib.</p>'
            },
            {
                'url': 'https://softpython.readthedocs.io/it/latest/exercises/visualization/visualization-solution.html',
                'contenuto' : '<h2>Visualizzazione</h2> <p>Excel ci permette di creare molti tipi di visualizzazione ma a volte è limitato, perciò useremo Python facendo grafici in Matplotlib. Matplotlib è una libreria eccezionale che consente di creare qualunque visualizzazione desideriamo. Procediamo dunque a importare matplotlib in Python:</p>'
            }
        ]

Supponiamo che tutto il web sia costituito dalle due pagine precedenti. Supponiamo anche che le pagine non cambino mai nel tempo. Abbiamo creato il nostro primo rudimentale _database_ del web!


#### Implementiamo la funzione di ricerca

Ma come possiamo implementare la ricerca? 


In [3]:


def ricerca(stringa):
    risultati = []             # all'inizio non sappiamo quali pagine contengono le parole cercate
    for pagina in pagine:                    # scorri le pagine
        if stringa in pagina['contenuto']:   # se la stringa cercata è nel contenuto della pagina ...
            risultati.append(pagina)         # aggiungi la pagina ai risultati collezionati finora
    return risultati                         # ritorna le pagine collezionate finora
    

Proviamola con una sola parola chiave `grafici`, dovrebbe ritornarci la pagina 1-esima sulla visualizzazione: 

In [4]:
ricerca("grafici")

[{'contenuto': '<h2>Visualizzazione</h2> <p>Excel ci permette di creare molti tipi di visualizzazione ma a volte è limitato, perciò useremo Python facendo grafici in Matplotlib. Matplotlib è una libreria eccezionale che consente di creare qualunque visualizzazione desideriamo. Procediamo dunque a importare matplotlib in Python:</p>',
  'url': 'https://softpython.readthedocs.io/it/latest/exercises/visualization/visualization-solution.html'}]

Funziona ! Baldanzosi col risultato del nostro primo esperimento, proviamo a cerca la stringa completa che ci eravamo prefissati all'inizio, `grafici con Matplotlib`:

Nulla ! Come mai? 

**DOMANDA**: Prima di proseguire, osserva bene il testo semplificato della pagina Visualizzazione e prova a dare una risposta. Ci sono due problemi principali. Come li risolveresti ? 


**RISPOSTA**: 

Se osserviamo bene il testo della pagina Visualizzazione, vediamo che riporta in fondo `"come fare grafici in Matplotlib"`

Saltano fuori diversi problemi: 

1. il nostro  utente sta cercando `"grafici con matplotlib"` usando `"con"` ma nel testo della pagina c'è scritto `"grafici in Matplotlib"` con `"in"`
2. l'utente cerca `"matplotlib"` in minuscolo, ma nella pagina è riportato `"Matplotlib"` in maiuscolo.


**DOMANDA**: Come risolveresti i problemi qua sopra?

**RISPOSTA**: 
    
1. Preposizioni come "con", "in", "di", "fra" etc molto spesso non sono rilevanti ai fini di una ricerca. Nel gergo dell'Information retrieval, vengono chiamate [stopwords](http://comunicaresulweb.com/seo/stop-words-italiane/). Per garantire una ricerca più efficace, potresti eliminarle dalla stringa di ricerca dell'utente. Ai fini della ricerca, forse avrebbe senso eliminarle anche dal testo delle pagine ?
2. Per risolvere problemi di maiuscole/minuscole, spesso conviene convertire la stringa di ricerca tutta in minuscolo, e fare lo stesso con il testo della pagina. Per farlo, puoi usare il metodo `lower()` delle stringhe. **NOTA**: `lower()`, come _tutti_ i metodi delle stringhe, ritorna una **nuova** stringa e non modifica l'originale !!


### Una ricerca più efficace

Accediamo al contenuto della prima pagina:

In [5]:
pagine[0]["contenuto"]

'<h2>Presentazione</h2><p>Il corso SoftPython fornisce un’introduzione al processamento dati usando Python, un linguaggio di programmazione popolare sia nell’industria che nell’ambito della ricerca. Fra le varie cose tratteremo Matplotlib.</p>'

Se cerco `"matplotlib"` in maiuscolo non lo troverò:

In [6]:
"matplotlib" in pagine[0]["contenuto"]

False

ma se metto tutto in minuscolo:

<div class="alert alert-warning">

**ATTENZIONE**: `lower()`, come _tutti_ i metodi delle stringhe, ritorna una **nuova** stringa e non modifica loriginale !!

</div>


In [7]:
pagine[0]["contenuto"].lower()

'<h2>presentazione</h2><p>il corso softpython fornisce un’introduzione al processamento dati usando python, un linguaggio di programmazione popolare sia nell’industria che nell’ambito della ricerca. fra le varie cose tratteremo matplotlib.</p>'

adesso, `matplotlib` in minuscolo verrà trovato nella stringa generata dall'espressione qua sopra: 

In [8]:
"matplotlib" in pagine[0]["contenuto"].lower()

True

Per essere sempre sicuri di trovare quello che l'utente cerca, conviene chiamare `.lower()` anche sulla stringa dell'utente:

In [9]:
"matplotlib".lower() in pagine[0]["contenuto"].lower()

True

In [10]:
"Matplotlib".lower() in pagine[0]["contenuto"].lower()

True

Proviamo a scrivere una ricerca più furba `ricerca2` che applica quanto visto sopra:

In [11]:
def ricerca2(stringa):
    risultati = []
    for pagina in pagine:
        if stringa.lower() in pagina['contenuto'].lower():
            risultati.append(pagina)
    return risultati

In [12]:
ricerca2("matplotlib")

[{'contenuto': '<h2>Presentazione</h2><p>Il corso SoftPython fornisce un’introduzione al processamento dati usando Python, un linguaggio di programmazione popolare sia nell’industria che nell’ambito della ricerca. Fra le varie cose tratteremo Matplotlib.</p>',
  'url': 'http://softpython.readthedocs.org'},
 {'contenuto': '<h2>Visualizzazione</h2> <p>Excel ci permette di creare molti tipi di visualizzazione ma a volte è limitato, perciò useremo Python facendo grafici in Matplotlib. Matplotlib è una libreria eccezionale che consente di creare qualunque visualizzazione desideriamo. Procediamo dunque a importare matplotlib in Python:</p>',
  'url': 'https://softpython.readthedocs.io/it/latest/exercises/visualization/visualization-solution.html'}]

In [13]:
ricerca2("Matplotlib")

[{'contenuto': '<h2>Presentazione</h2><p>Il corso SoftPython fornisce un’introduzione al processamento dati usando Python, un linguaggio di programmazione popolare sia nell’industria che nell’ambito della ricerca. Fra le varie cose tratteremo Matplotlib.</p>',
  'url': 'http://softpython.readthedocs.org'},
 {'contenuto': '<h2>Visualizzazione</h2> <p>Excel ci permette di creare molti tipi di visualizzazione ma a volte è limitato, perciò useremo Python facendo grafici in Matplotlib. Matplotlib è una libreria eccezionale che consente di creare qualunque visualizzazione desideriamo. Procediamo dunque a importare matplotlib in Python:</p>',
  'url': 'https://softpython.readthedocs.io/it/latest/exercises/visualization/visualization-solution.html'}]

### Performance

**DOMANDA**: 

- Se abbiamo miliardi di pagine, conviene cercarle in una lista? Se la stringa che cerchiamo sta nell'ultima pagina, l'algoritmo di ricerca quante pagine guarda prima di trovare quella d'interesse?
- Che strutture dati alternative potremmo usare per la ricerca? 

**RISPOSTA**: Ci converra _indicizzare_ le pagine, cioè avere un indice che ci dice per ogni parola di ricerca in quali pagine possiamo trovarla. 

* Per implementare questa corrispondenza in Python possiamo usare i dizionari. 
* Per convenienza, ci conviene indicare le pagine con il numero che abbiamo usato quando le abbiamo messe nella listona. 
* Di nuovo per convenienza, metteremo solo parole in minuscolo.

Quindi potrebbe essere  una cosa del genere: 

```
{
    "linguaggio": [0],    # la stringa "linguaggio" sta alla pagina zeresima della listona
    "matplotlib" : [0,1], # la stringa "matplotlib" sta nelle pagine zeresima e unesima  della listona
    "importare" : [1]     # "importare" sta solo nella pagina sulla visualizzazione
}
```


### Ordinamento

Osserva di nuovo bene il testo delle pagine. Noti delle particolarità / differenze tra le due pagine ?


In [14]:
pagine = [
            {
                'url': 'http://softpython.readthedocs.org',
                'contenuto' : '<h2>Presentazione</h2><p>Il corso SoftPython fornisce un’introduzione al processamento dati usando Python, un linguaggio diprogrammazione popolare sia nell’industria che nell’ambito della ricerca. Fra le varie cose tratteremo Matplotlib.</p>'
            },
            {
                'url': 'https://softpython.readthedocs.io/it/latest/exercises/visualization/visualization-solution.html',
                'contenuto' : '<h2>Visualizzazione</h2> <p>Excel ci permette di creare molti tipi di visualizzazione ma a volte è limitato, perciò useremo Python facendo grafici in Matplotlib. Matplotlib è una libreria eccezionale che consente di creare qualunque visualizzazione desideriamo. Procediamo dunque a importare matplotlib in Python:</p>'
            }
        ]

Se l'utente cerca solo la stringa `"matplotlib"`, visto che è presente in entrambe le pagine  ci aspettiamo che vengano ritornate entrambe:

In [15]:
ricerca2("matplotlib")

[{'contenuto': '<h2>Presentazione</h2><p>Il corso SoftPython fornisce un’introduzione al processamento dati usando Python, un linguaggio diprogrammazione popolare sia nell’industria che nell’ambito della ricerca. Fra le varie cose tratteremo Matplotlib.</p>',
  'url': 'http://softpython.readthedocs.org'},
 {'contenuto': '<h2>Visualizzazione</h2> <p>Excel ci permette di creare molti tipi di visualizzazione ma a volte è limitato, perciò useremo Python facendo grafici in Matplotlib. Matplotlib è una libreria eccezionale che consente di creare qualunque visualizzazione desideriamo. Procediamo dunque a importare matplotlib in Python:</p>',
  'url': 'https://softpython.readthedocs.io/it/latest/exercises/visualization/visualization-solution.html'}]

**DOMANDA**: Pensiamo all'rodine delle pagine. E' il migliore che possiamo fornire all'utente? Secondo te, qual'è la pagina che contiene più informazione riguardo Matplotlib e che andrebbe messa per prima? 

**RISPOSTA**: 
    
Ovviamente, noi umani sappiamo la pagina specifica sulla Visualizzazione sarà quella preferita dall'utente. Per capirlo automaticamente con un algoritmo possiamo notare che la frequenza di `"Matplotlib"` è mggiore nella seconda pagina rispetto alla prima. Possiamo farci dire la frequenza da Python con il comando `count`:

In [16]:
pagine[0]["contenuto"].lower().count("Matplotlib".lower())

1

In [17]:
pagine[1]["contenuto"].lower().count("Matplotlib".lower())

3

**DA FARE**: Assegna ad ogni pagina un valore di rilevanza (_relevance_)  ,  che permetta di determinare l'ordine (_rank_) in cui ritornare le pagine. Le pagine con rilevanza più alta devono essere presentate per prime nella lista ritornata

In [ ]:
# scrivi qui